# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
import os
import joblib

from azureml.train.automl import AutoMLConfig
from pprint import pprint 
from azureml.core import Model

In [4]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'udacity-capstone'

experiment=Experiment(ws, experiment_name)

In [8]:
compute_target = ComputeTarget(workspace=ws, name="cpu-sbinstance")

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

The below dataset is a hearth failure clinical record from Kaggle as mentioned in the project description. The task is to predict whether the heart will fail or not?

In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

In [19]:
dataset = TabularDatasetFactory().from_delimited_files("https://raw.githubusercontent.com/bhupendrasolanki/udacity-capstone/main/heart_failure_clinical_records_dataset.csv") ### YOUR CODE HERE ###
df = dataset.to_pandas_dataframe().dropna()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00
mean,60.83,0.43,581.84,0.42,38.08,0.35,263358.03,1.39,136.63,0.65,0.32,130.26,0.32
std,11.89,0.50,970.29,0.49,11.83,0.48,97804.24,1.03,4.41,0.48,0.47,77.61,0.47
min,40.00,0.00,23.00,0.00,14.00,0.00,25100.00,0.50,113.00,0.00,0.00,4.00,0.00
25%,51.00,0.00,116.50,0.00,30.00,0.00,212500.00,0.90,134.00,0.00,0.00,73.00,0.00
50%,60.00,0.00,250.00,0.00,38.00,0.00,262000.00,1.10,137.00,1.00,0.00,115.00,0.00
75%,70.00,1.00,582.00,1.00,45.00,1.00,303500.00,1.40,140.00,1.00,1.00,203.00,1.00
max,95.00,1.00,7861.00,1.00,80.00,1.00,850000.00,9.40,148.00,1.00,1.00,285.00,1.00


In [9]:
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [13]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
        task='classification',
        compute_target=compute_target,
        training_data=dataset,
        label_column_name='DEATH_EVENT',
        n_cross_validations=5,
        **automl_settings
)

In [14]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-capstone,AutoML_c8cfe321-8de0-4ede-9550-cbd201a5f3e7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [15]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-capstone,AutoML_c8cfe321-8de0-4ede-9550-cbd201a5f3e7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

{'runId': 'AutoML_c8cfe321-8de0-4ede-9550-cbd201a5f3e7',
 'target': 'cpu-sbinstance',
 'status': 'Completed',
 'startTimeUtc': '2024-02-09T05:31:33.0461Z',
 'endTimeUtc': '2024-02-09T05:47:11.311541Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cpu-sbinstance',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-capstone","subscription_id":"6b4af8be-9931-443e-90f6-c4c34a1f9737","resource_group":"aml-quickstarts-252188","workspace_name":"quick-starts-ws-252188","region":"westus2","compute_target":"cpu-sbinstance",

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [17]:
# Get best run and model
best_run, fitted_model = remote_run.get_output()

# Print the best run
print(best_run)
# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()

# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

Run(Experiment: udacity-capstone,
Id: AutoML_c8cfe321-8de0-4ede-9550-cbd201a5f3e7_55,
Type: azureml.scriptrun,
Status: Completed)
AUC_macro 0.9192687338501292
AUC_weighted 0.9192687338501292
log_loss 0.363852781580171
f1_score_micro 0.8930508474576271
average_precision_score_weighted 0.9284507367470065
precision_score_macro 0.897721055460622
accuracy 0.8930508474576271
norm_macro_recall 0.730952380952381
f1_score_macro 0.8701440289333456
precision_score_micro 0.8930508474576271
recall_score_weighted 0.8930508474576271
f1_score_weighted 0.8898830531029361
average_precision_score_micro 0.9225193776148423
recall_score_micro 0.8930508474576271
AUC_micro 0.9197958600657538
weighted_accuracy 0.9115054180922364
precision_score_weighted 0.9065511374387738
matthews_correlation 0.7607915062466252
average_precision_score_macro 0.9026959391883228
recall_score_macro 0.8654761904761905
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_c8cfe321-8de0-4ede-9550-cbd201a5f3e7_55/confusion_matri

In [18]:
#TODO: Save the best model
#TODO: Save the best model
best_model = best_run.register_model(model_path='outputs/model.pkl', model_name='automl_best_model_capstone',
                        properties={'Accuracy': best_run_metrics['accuracy']})

print(best_model)

Model(workspace=Workspace.create(name='quick-starts-ws-252188', subscription_id='6b4af8be-9931-443e-90f6-c4c34a1f9737', resource_group='aml-quickstarts-252188'), name=automl_best_model_capstone, id=automl_best_model_capstone:1, version=1, tags={}, properties={'Accuracy': '0.8930508474576271'})


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [21]:
import os
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/cpu-sbinstance/code/Users/odl_user_252188'

In [20]:
# Download scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoreScript.py')

# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

TODO: In the cell below, send a request to the web service you deployed to test it.

In [22]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='scoreScript.py',
                                    environment=best_run.get_environment())

# deploy
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "myservice", [best_model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

print(service.scoring_uri)

print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-02-09 06:56:57+00:00 Creating Container Registry if not exists.
2024-02-09 06:56:57+00:00 Registering the environment.
2024-02-09 06:56:58+00:00 Use the existing image.
2024-02-09 06:56:58+00:00 Submitting deployment to compute.
2024-02-09 06:57:05+00:00 Checking the status of deployment myservice..
2024-02-09 06:59:09+00:00 Checking the status of inference endpoint myservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://7fec2b12-077a-4343-828a-5bdc5f9b720e.westus2.azurecontainer.io/score
http://7fec2b12-077a-4343-828a-5bdc5f9b720e.westus2.azurecontainer.io/swagger.json


TODO: In the cell below, print the logs of the web service and delete the service

In [24]:
import json

#Import test data
test_df = df.sample(5) # data is the pandas dataframe of the original data
label_df = test_df.pop('DEATH_EVENT')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)

{"data": [{"age": 65.0, "anaemia": 1, "creatinine_phosphokinase": 160, "diabetes": 1, "ejection_fraction": 20, "high_blood_pressure": 0, "platelets": 327000.0, "serum_creatinine": 2.7, "serum_sodium": 116, "sex": 0, "smoking": 0, "time": 8}, {"age": 50.0, "anaemia": 0, "creatinine_phosphokinase": 369, "diabetes": 1, "ejection_fraction": 25, "high_blood_pressure": 0, "platelets": 252000.0, "serum_creatinine": 1.6, "serum_sodium": 136, "sex": 1, "smoking": 0, "time": 90}, {"age": 50.0, "anaemia": 0, "creatinine_phosphokinase": 124, "diabetes": 1, "ejection_fraction": 30, "high_blood_pressure": 1, "platelets": 153000.0, "serum_creatinine": 1.2, "serum_sodium": 136, "sex": 0, "smoking": 1, "time": 32}, {"age": 55.0, "anaemia": 0, "creatinine_phosphokinase": 47, "diabetes": 0, "ejection_fraction": 35, "high_blood_pressure": 1, "platelets": 173000.0, "serum_creatinine": 1.1, "serum_sodium": 137, "sex": 1, "smoking": 0, "time": 79}, {"age": 55.0, "anaemia": 0, "creatinine_phosphokinase": 7861

In [25]:
import requests # Used for http post request

# Set the content type
headers = {'Content-type': 'application/json'}


response = requests.post(service.scoring_uri, test_sample, headers=headers)

In [26]:
print(response.text)

"{\"result\": [1, 0, 1, 0, 1]}"


In [27]:
print(service.get_logs())

2024-02-09T06:59:01,660854300+00:00 - rsyslog/run 
2024-02-09T06:59:01,685283700+00:00 - gunicorn/run 
2024-02-09T06:59:01,692471600+00:00 | gunicorn/run | 
2024-02-09T06:59:01,694450400+00:00 | gunicorn/run | ###############################################
2024-02-09T06:59:01,695114800+00:00 - nginx/run 
2024-02-09T06:59:01,696293600+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-02-09T06:59:01,705299100+00:00 | gunicorn/run | ###############################################
2024-02-09T06:59:01,709339900+00:00 | gunicorn/run | 
2024-02-09T06:59:01,725235800+00:00 | gunicorn/run | 
2024-02-09T06:59:01,737299700+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2
2024-02-09T06:59:01,740571000+00:00 | gunicorn/run | 
2024-02-09T06:59:01,742645900+00:00 | gunicorn/run | 
2024-02-09T06:59:01,752472300+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

In [28]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
